# Tutorial: Numerical implementation of neutrino leakage scheme

## Author: Leo Werneck

<a id=toc></a>

# Table of Contents
$$\label{toc}$$

1. [Step 1](#calc_leak_f90): `calc_leak.F90` 
    1. [Step 1.a](#zelmanileak_calcleak): `ZelmaniLeak_CalcLeak()`
    1. [Step 1.b](#calc_leak): `calc_leak()`
    1. [Step 1.c](#linterp2dn): `linterp2Dn()`
    1. [Step 1.d](#linterp3dn): `linterp3Dn()`
1. [Step 2](#calc_taus_f90): calc_taus.F90
    1. [Step 2.a](#get_fermi_integral_leak): `get_fermi_integral_leak()`
1. [Step 3](#ye_of_rho_f90): `ye_of_rho.F90` <font color='green'>**FINISHED**</font>
    1. [Step 3.a](#zelmanileak_ye_of_rho): `ZelmaniLeak_ye_of_rho()`
    1. [Step 3.b](#fit_ye): `fit_ye()`
    1. [Step 3.c](#zelmanileak_calc_entropy): `ZelmaniLeak_calc_entropy()`
1. [Step 4](#ye_of_rho_from_profile_f90): `ye_of_rho_from_profile.F90` <font color='green'>**FINISHED**</font>
    1. [Step 4.a](#yeofrho): `yeofrho()`
    1. [Step 4.b](#readprofile_ye): `readprofile_ye()`
    1. [Step 4.c](#setuprho_ye): `setuprho_ye()`
    1. [Step 4.d](#setupye_ye): `setupye_ye()`
    1. [Step 4.e](#map_find_index_ye): `map_find_index_ye()`
    1. [Step 4.f](#linterp_ye): `linterp_ye()`

<a id=calc_leak_f90></a>

# Step 1: `calc_leak.F90` \[Back to [Top](#toc)\]
$$\label{calc_leak_f90}$$

$$
\newcommand{\RR}{\mathcal{R}}
\newcommand{\QQ}{\mathcal{Q}}
\newcommand{\nui}{\nu_{i}}
\newcommand{\anui}{\bar{\nu}_{i}}
\newcommand{\nux}{\nu_{\rm x}}
\newcommand{\anux}{\bar{\nu}_{\rm x}}
\newcommand{\nue}{\nu_{\rm e}}
\newcommand{\anue}{\bar{\nu}_{\rm e}}
\newcommand{\mb}{m_{\rm b}}
\newcommand{\me}{m_{\rm e}}
\newcommand{\nb}{n_{\rm b}}
\newcommand{\ne}{n_{\rm e}}
\newcommand{\rhob}{\rho_{\rm b}}
\newcommand{\ye}{Y_{\rm e}}
\newcommand{\Fdual}{{}^{*}\!F}
\newcommand{\ee}{e^{-}}
\newcommand{\ae}{e^{+}}
$$

<a id=zelmanileak_calcleak></a>

## Step 1.a: `ZelmaniLeak_CalcLeak()` \[Back to [Top](#toc)\]
$$\label{zelmanileak_calcleak}$$

<a id=calc_leak></a>

## Step 1.b: `calc_leak()` \[Back to [Top](#toc)\]
$$\label{calc_leak}$$

<a id=linterp2dn></a>

## Step 1.c: `linterp2Dn()` \[Back to [Top](#toc)\]
$$\label{linterp2dn}$$

This function performs [bilinear interpolation](https://en.wikipedia.org/wiki/Bilinear_interpolation) of a function $f$. Let $Q_{ij} = (x_{i},y_{j})$. The function returns,

$$
f_{\rm out}(x,y) = \frac{f_{\rm in}(Q_{11})(x_{2}-x)(y_{2}-y) + f_{\rm in}(Q_{21})(x-x_{1})(y_{2}-y) + f_{\rm in}(Q_{12})(x_{2}-x)(y-y_{1}) + f_{\rm in}(Q_{22})(x-x_{1})(y-y_{1})}{(x_{2}-x_{1})(y_{2}-y_{1})}.
$$

<a id=linterp3dn></a>

## Step 1.d: `linterp3Dn()` \[Back to [Top](#toc)\]
$$\label{linterp3dn}$$

This function performs [trilinear interpolation](https://en.wikipedia.org/wiki/Trilinear_interpolation) of a function $f$. Let $Q_{ijk} = (x_{i},y_{j},z_{k})$. The function returns,

$$
\begin{split}
f_{\rm out}(x,y,z) = \left[(x_{2}-x_{1})(y_{2}-y_{1})(z_{2}-z_{1})\right]^{-1}
\big[&f_{\rm in}(Q_{111})(x_{2}-x)(y_{2}-y)(z_{2}-z)\\
+&f_{\rm in}(Q_{211})(x-x_{1})(y_{2}-y)(z_{2}-z)\\
+&f_{\rm in}(Q_{121})(x_{2}-x)(y-y_{1})(z_{2}-z)\\
+&f_{\rm in}(Q_{112})(x_{2}-x)(y_{2}-y)(z-z_{1})\\
+&f_{\rm in}(Q_{221})(x-x_{1})(y-y_{1})(z_{2}-z)\\
+&f_{\rm in}(Q_{212})(x-x_{1})(y_{2}-y)(z-z_{1})\\
+&f_{\rm in}(Q_{122})(x_{2}-x)(y-y_{1})(z-z_{1})\\
+&f_{\rm in}(Q_{222})(x-x_{1})(y-y_{1})(z-z_{1})
\big]
\end{split}.
$$

<a id=calc_taus_f90></a>

# Step 2: `calc_taus.F90` \[Back to [Top](#toc)\]
$$\label{calc_taus_f90}$$

This file contains 3 functions:

* `ZelmaniLeak_CalcTau()`
* `calc_taus()`
* `get_fermi_integral_leak()`

<a id=get_fermi_integral_leak></a>

## Step 2.a: `get_fermi_integral_leak()` \[Back to [Top](#toc)\]
$$\label{get_fermi_integral_leak}$$

Function `get_fermi_integral_leak()` contains hard-coded expressions for the Fermi-Dirac integrals

$$
F_{N}(\eta) = \int_{0}^{\infty}\frac{x^{N}dx}{e^{x-\eta}+1},
$$

for $N=0,\ldots,5$, following [Takahashi *et al.* (1978)](http://adsabs.harvard.edu/pdf/1978A%26A....67..185T). For $\eta>10^{-3}$, the function returns

$$
\begin{align}
F_{0}(\eta) &= \log_{10}\left(1+e^{\eta}\right),\\
F_{1}(\eta) &= \frac{\eta^{2}/2 + 1.6449}{1+\exp(-1.6855\eta)},\\
F_{2}(\eta) &= \frac{\eta^{3}/3 + 3.2899\eta}{1-\exp(-1.8246\eta)},\\
F_{3}(\eta) &= \frac{\eta^{4}/4 + 4.9348\eta^{2} + 11.3644}{1+\exp(-1.9039\eta)},\\
F_{4}(\eta) &= \frac{\eta^{5}/5 + 6.5797\eta^{3} + 45.4576\eta}{1-\exp(-1.9484\eta)},\\
F_{5}(\eta) &= \frac{\eta^{6}/6 + 8.2247\eta^{4} + 113.6439\eta^{2} + 236.5323}{1+exp(-1.9727\eta)},
\end{align}
$$

while for $\eta<10^{-3}$ it returns

$$
\begin{align}
F_{0}(\eta) &= \log_{10}\left(1+e^{\eta}\right),\\
F_{1}(\eta) &= e^{\eta}\left[1 + 0.2159\exp(0.8857\eta)\right]^{-1},\\
F_{2}(\eta) &= 2e^{\eta}\left[1 + 0.1095\exp(0.8908\eta)\right]^{-1},\\
F_{3}(\eta) &= 6e^{\eta}\left[1 + 0.0559\exp(0.9069\eta)\right]^{-1},\\
F_{4}(\eta) &= 24e^{\eta}\left[1 + 0.0287\exp(0.9257\eta)\right]^{-1},\\
F_{5}(\eta) &= 120e^{\eta}\left[1 + 0.0147\exp(0.9431\eta)\right]^{-1}.
\end{align}
$$

<a id=ye_of_rho_f90></a>

# Step 3: `ye_of_rho.F90` \[Back to [Top](#toc)\]
$$\label{ye_of_rho_f90}$$

<a id=zelmanileak_ye_of_rho></a>

## Step 3.a: `ZelmaniLeak_ye_of_rho()` \[Back to [Top](#toc)\]
$$\label{zelmanileak_ye_of_rho}$$

Driver function. It selects whether to perform the fitting described in [Step 3.b](#fit_ye) or a linear interpolation of the tabulated values $\bar{\ye}(\rhob)$.

<a id=fit_ye></a>

## Step 3.b: `fit_ye()` \[Back to [Top](#toc)\]
$$\label{fit_ye}$$

This functions determines $\bar{\ye}(\rhob)$ using the fitting function of [Liebendörfer (2005)](https://arxiv.org/pdf/astro-ph/0504072.pdf), i.e.,

$$
\bar{\ye}(\rhob) = \bar{\ye}[x(\rhob)] = \frac{1}{2}\left(Y_{2}+Y_{1}\right) + \frac{x}{2}\left(Y_{2}-Y_{1}\right) + Y_{\rm c}\left[1 - |x| + 4|x|\left(|x|-\frac{1}{2}\right)\left(|x|-1\right)\right],
$$

where

$$
x(\rhob) = \max\left[-1,\min\left(1,\frac{2\log\rhob-\log\rho_{2}-\log\rho_{1}}{\log\rho_{2}-\log\rho_{1}}\right)\right].
$$

According to Table I in [Liebendörfer (2005)](https://arxiv.org/abs/astro-ph/0504072), we have:

* Model N13 (simplified, Newtonian gravity): $Y_{\rm c}=0.035$, $Y_{1}=0.5$, $Y_{2}=0.285$, $\rho_{1}=2\times10^{7}\ {\rm g\,cm^{-3}}$, $\rho_{2}=2\times10^{13}\ {\rm g\,cm^{-3}}$;
* Model G15 (complete, general relativity): $Y_{\rm c}=0.035$, $Y_{1}=0.5$, $Y_{2}=0.278$, $\rho_{1}=3\times10^{7}\ {\rm g\,cm^{-3}}$, $\rho_{2}=2\times10^{13}\ {\rm g\,cm^{-3}}$.

These parameters can be specified at the parfile level using:

* $\texttt{yeofrho}\_\texttt{rho1} \leftrightarrow \rho_{1}$, with $[\rho_{1}] = {\rm g\,cm^{-3}}$;
* $\texttt{yeofrho}\_\texttt{rho2} \leftrightarrow \rho_{2}$, with $[\rho_{2}] = {\rm g\,cm^{-3}}$;
* $\texttt{yeofrho}\_\texttt{ye1} \leftrightarrow Y_{1}$;
* $\texttt{yeofrho}\_\texttt{ye2} \leftrightarrow Y_{2}$;
* $\texttt{yeofrho}\_\texttt{yec} \leftrightarrow Y_{\rm c}$,

and the default values are those of model G15.

<a id=zelmanileak_calc_entropy></a>

## Step 3.c: `ZelmaniLeak_calc_entropy()` \[Back to [Top](#toc)\]
$$\label{zelmanileak_calc_entropy}$$

Loops over the grid and computes the entropy using $\texttt{EOS}\_\texttt{Omni}$. According to the comments, this function is deprecated (no longer used).

<a id=ye_of_rho_from_profile_f90></a>

# Step 4: `ye_of_rho_from_profile.F90` \[Back to [Top](#toc)\]
$$\label{ye_of_rho_from_profile_f90}$$

<a id=yeofrho></a>

## Step 4.a: `yeofrho()` \[Back to [Top](#toc)\]
$$\label{yeofrho}$$

Interpolates tabulated $\bar\ye$, returning $\ye$ at the input density $\rhob$.

<a id=readprofile_ye></a>

## Step 4.b: `readprofile_ye()` \[Back to [Top](#toc)\]
$$\label{readprofile_ye}$$

Read in tabulated $\bar\ye$ from input file.

<a id=setuprho_ye></a>

## Step 4.c: `setuprho_ye()` \[Back to [Top](#toc)\]
$$\label{setuprho_ye}$$

Sets up an logarithmic evenly spaced array $\log\rhob$ ranging from $\log\rho_{\min}$ to $\log\rho_{\max}$. The number of points in the array, as well as $\rho_{\min}$ and $\rho_{\max}$, are function inputs.

<a id=setupye_ye></a>

## Step 4.d: `setupye_ye()` \[Back to [Top](#toc)\]
$$\label{setupye_ye}$$

This function takes as input an array of density values and constructs the corresponding array $\ye(\rhob)$ using the table $\bar{\ye}(\rhob)$.

<a id=map_find_index_ye></a>

## Step 4.e: `map_find_index_ye()` \[Back to [Top](#toc)\]
$$\label{map_find_index_ye}$$

Given an input array $Y$ and a value $y$, the function searches for the element in $Y$ which is cloest to $y$ and returns its index. It finds the index by using the [bisection method](https://en.wikipedia.org/wiki/Bisection_method).

<a id=linterp_ye></a>

## Step 4.f: `linterp_ye()` \[Back to [Top](#toc)\]
$$\label{linterp_ye}$$

Performs [linear interpolation](https://en.wikipedia.org/wiki/Linear_interpolation), i.e.,

$$
y(x) = y_{1} + \left(\frac{y_{2}-y_{1}}{x_{2}-x_{1}}\right)(x-x_{1}),
$$

where $(x_{1},x_{2},y_{1},y_{2})$ are inputs, the point of interest is $x$ and the output is stored in $y$.

<a id=calc_grr_cc></a>

# Step 5: `calc_grr.cc` \[Back to [Top](#toc)\]
$$\label{calc_grr_cc}$$

<a id=zltau_calc_grr></a>

## Step 5.a: `ZLtau_calc_grr()` \[Back to [Top](#toc)\]
$$\label{zltau_calc_grr}$$

This function takes as input $\gamma_{ij}$ and computes $\gamma_{rr}$, i.e., it performs a base transformation

$$
\gamma_{ij}^{\rm Sph} = \frac{\partial x^{k}_{\rm Cart}}{\partial x^{i}_{\rm Sph}}\frac{\partial x^{l}_{\rm Cart}}{\partial x^{j}_{\rm Sph}}\gamma_{kl}^{\rm Cart},
$$

where only the $rr$-component of $\gamma_{ij}^{\rm Sph}$ is computed, i.e.,

$$
\gamma_{rr}^{\rm Sph} = \frac{\partial x^{k}_{\rm Cart}}{\partial r}\frac{\partial x^{l}_{\rm Cart}}{\partial r}\gamma_{kl}^{\rm Cart}.
$$

For standard [spherical coordinates](https://en.wikipedia.org/wiki/Spherical_coordinate_system) we have

$$
\begin{aligned}
x &= r\sin\theta\cos\phi,\\
y &= r\sin\theta\sin\phi,\\
z &= r\cos\theta,
\end{aligned}
$$

and thus

$$
\frac{\partial x^{i}_{\rm Cart}}{\partial r} = \bigl(\sin\theta\cos\phi,\sin\theta\sin\phi,\cos\theta\bigr).
$$

The implemented expression is thus

$$
\begin{split}
\gamma_{rr} &= \left(\sin\theta\cos\phi\right)^{2}\gamma_{xx} + \left(\sin\theta\sin\phi\right)^{2}\gamma_{yy} + \cos^{2}\theta\gamma_{zz}\\
&+ 2\left[\sin^{2}\theta\cos\phi\sin\phi\gamma_{xy} + \sin\theta\cos\theta\cos\phi\gamma_{xz} + \sin\theta\cos\theta\sin\phi\gamma_{yz}\right].
\end{split}
$$

The function also accounts for small values of $r$ and the $xy$-plane by setting $\theta=0$ and $\phi=0$ manually in those cases, otherwise it uses the [standard formulae]()

$$
\begin{align}
\sin\theta &= \frac{\sqrt{x^{2}+y^{2}}}{r},\\
\cos\theta &= \frac{z}{r},\\
\sin\phi &= \frac{y}{\sqrt{x^{2}+y^{2}}},\\
\cos\phi &= \frac{x}{\sqrt{x^{2}+y^{2}}}.
\end{align}
$$

<a id=tau_cc></a>

# Step 6: `tau.cc` \[Back to [Top](#toc)\]
$$\label{tau_cc}$$

<a id=zltau_setup_local></a>

## Step 6.a: `ZLtau_setup_local()`
$$\label{zltau_setup_local}$$

Initializes gridfunctions for optical depth, integrated luminosities, and neutrino luminosities to zero.

<a id=zltau_register></a>

## Step 6.b: `ZLtau_register()`
$$\label{zltau_register}$$

Register gridfunctions for MoL using `MoLRegisterConstrained`.

<a id=zltau_setup></a>

## Step 6.c: `ZLtau_setup()`
$$\label{zltau_setup}$$

Sets up numerical grid for evaluation of optical depth.